In [2]:
##modules
#%matplotlib widget
#%matplotlib inline
#
#%matplotlib qt
import mne
import numpy as np
import matplotlib
# Establecer un backend interactivo, como 'Qt5Agg', 'GTK3Agg', etc.
# Esto depende de los backends disponibles en tu sistema.

import matplotlib.pyplot as plt

matplotlib.use('TkAgg')  # Asegúrate de que este backend está instalado.

import pandas as pd 
import os
import sys

from mne.preprocessing import ICA, corrmap, create_ecg_epochs, create_eog_epochs

from os.path import join as pathjoin
from time import time

from pathlib import Path

from autoreject import AutoReject

In [3]:
# Variables path
layer_script = "block"

#process=""
disco="g"

subj = "S01"

#subj = sys.argv[1] ## name of participant list

# Carpeta general
datadir = Path(f"{disco}:\\PROYECTO_SELF")

# #carpetas generales de datos
brain_vision_dir = datadir /"Data_self"/"Export_acw_Brain_Vision"
fif_data = datadir /"Data_self"/ "fif_data" 


#carpeta analysis
output_analysis= datadir / "output_analysis"

acw_analysis_path = output_analysis / "acw"
acw_analysis_path.mkdir(parents=True, exist_ok=True)

# # Carpeta de preprocesado
# output_preproc = datadir / "output_preproc"

# preproc_path = output_preproc / f"preproc_{layer_script}"
# preproc_path.mkdir(parents=True, exist_ok=True)
 
# # Carpeta de epocas "sucias"
# epochs_path = preproc_path / f"epochs_{layer_script}"
# epochs_path.mkdir(parents=True, exist_ok=True) 

# # Carpeta de ICA
# ICA_path = preproc_path / f"ICA_{layer_script}"
# ICA_path.mkdir(parents=True, exist_ok=True)

# # Épocas limpias
# epochs_clean_path = preproc_path / f"epochs_clean_{layer_script}"
# epochs_clean_path.mkdir(parents=True, exist_ok=True)

# #epocas epochs
# epochs_path = Path(preproc_path) / f"epochs_{layer_script}"
# epochs_path.mkdir(parents=True, exist_ok=True)
 

# # Definir la carpeta de output_source antes de usarla
# output_source = Path(r"g:\MOUS_204\output_source")

# source_path = output_source / f"source_{layer_script}"
# source_path.mkdir(parents=True, exist_ok=True)

# #raw_hsp es el raw con fiducials cargados
# raw_hsp_path = source_path / f"raw_hsp"
# raw_hsp_path.mkdir(parents=True, exist_ok=True)

# # Carpeta de forward solution
# fwd_path = source_path / f"fwd"
# fwd_path.mkdir(parents=True, exist_ok=True)

# # Carpeta de inverse solution
# inverse_path = source_path / f"inverse"
# inverse_path.mkdir(parents=True, exist_ok=True)


# mne.utils.set_config('SUBJECTS_DIR', r'\\wsl$\Ubuntu-20.04\usr\local\freesurfer\subjects', set_env=True)


In [173]:

#epochs
combinaciones = ["S1 - S4","S1 - S5", "S2 - S4", "S2 - S5", "S3 - S4", "S3 - S5"]

combinacion= combinaciones[1]

# Obtener todos los archivos y carpetas dentro de la ruta
subjects = list(fif_data.iterdir())

# Filtrar solo los nombres de sujetos (removiendo "_epochs")
subject_names = [subject.stem.replace("_epochs", "") for subject in subjects]

subj=subject_names[0]
subj


# # epochs_zinnen_path = epochs_clean_path / f"{subj}_epochs_zinnen_{type_script}"
# # epochs_path = epochs_clean_path / f"{subj}_epochs_{type_script}"

path_epochs= fif_data / f"{subj}_epochs" / f"{subj}_{combinacion}-epo.fif"

epochs = mne.read_epochs(path_epochs) 





Reading g:\PROYECTO_SELF\Data_self\fif_data\S01_epochs\S01_S1 - S5-epo.fif ...
    Found the data of interest:
        t =       0.00 ...    9198.00 ms
        0 CTF compensation matrices available
Not setting metadata
21 matching events found
No baseline correction applied
0 projection items activated


In [171]:
# aplicar la acf EN epochs

from statsmodels.tsa.stattools import acf
import numpy as np

import pandas as pd

def acf_epochs(epochs, isplot=False): 
    
    #import epochs object and get data ONLY ON MEG DATA
    data_epochs=epochs.copy().pick("eeg").get_data()

    #get duration
    duration= epochs.tmax - epochs.tmin
    #get sample frequency
    sfreq= epochs.info['sfreq']
    #get lags
    lags=duration*sfreq

    del epochs

    #list to store the values of acf, acw_50,acw_0 for each epoch for each sensor for each EPOCH
    acf_epoch_all = []

    #conjunto de todos los valroes de ACW_0-ACW_50 para cada sensor en cada epoca
    acw_50_epoch_all = []
    acw_0_epoch_all = []

    #promedio de valores de acw_50 y acw_0 para cada epoca (el promedio de todos los sensores en una epoca)
    acw_50_epoch_all_mean = []
    acw_0_epoch_all_mean = []

    #for each epoch
      
    for j in range(0,len(data_epochs)):
        #take data for each epoch
        data_epoch=data_epochs[j]

        #list to store the values of acf, acw_50,acw_0 for each SENSOR for each EPOCH

        acf_elect_all = []
        acf_elect_all_mean = []

        acw_50_elect_all = []
        acw_0_elect_all = []

    


        #calculus of value  for EACH SENSOR
        for i in range(0,len(data_epoch)):
            #lags se puede dejar por defecto porque te cogelos valores
            #hasta los que tiene sentido calcularlo, aunque yo voy a calcularlo para todos los lags
            #calculo de acf de sensor
            #alfa lo dejo endefault que es la confianza del 95% 

            acf_elect, qstat_vals, pvals = acf(
                data_epoch[i],
                adjusted=True,
                fft=True,
                qstat=True,
                nlags=lags
            )

            #conjunto de valores de acf en un electrodo en una epoca
            acf_elect_all.append(acf_elect)
            #promedio de valores de acf en un electrodo en una epoca
            acf_elect_all_mean.append(np.mean(acf_elect))


            #calculo de acw_50 y acw_0 de sensor for both in lags and in seconds
            #recuerda que acw_50 y acw_0 son valores unicos, no como acf que es un conjunto de valores
            acw_50_elect_lags = np.argmax(acf_elect <= 0.5)
            acw_50_elect_s = acw_50_elect_lags / sfreq
            acw_50_elect_all.append(acw_50_elect_s)


            acw_0_elect_lags = np.argmax(acf_elect <= 0)
            acw_0_elect_s = acw_0_elect_lags / sfreq
            acw_0_elect_all.append(acw_0_elect_s)


        #
        acw_50_epoch_all.append(acw_50_elect_all)
        acw_0_epoch_all.append(acw_0_elect_all)
        acf_epoch_all.append(acf_elect_all)


        acw_50_epoch_all_mean.append(np.mean(acw_50_elect_all))
        acw_0_epoch_all_mean.append(np.mean(acw_0_elect_all))
       
        #promedio de los electrodos en todas las epocas
        # Convertir la lista de listas en un array de NumPy para cálculos más eficientes
        acw_50_epoch_all_array = np.array(acw_50_epoch_all)  # (21 filas, 59 columnas)
        # Calcular el promedio a lo largo de las columnas (axis=0) → Promedio por electrodo en cada época
        promedio_ACW_50_electrodos = np.mean(acw_50_epoch_all_array, axis=0)

        acw_0_epoch_all_array = np.array(acw_0_epoch_all)  # (21 filas, 59 columnas)
        # Calcular el promedio a lo largo de las columnas (axis=0) → Promedio por electrodo en cada época       
        promedio_ACW_0_electrodos = np.mean(acw_0_epoch_all_array, axis=0)
    
    # #transformation of list to array
    # acf_elect_all=np.array(acf_elect_all)
    # acw_50_elect_all_s=np.array(acw_50_elect_all_s)
    # acw_0_elect_all_s=np.array(acw_0_elect_all_s)




    # # Crear un DataFrame con las columnas necesarias

    table_autocorrelation = pd.DataFrame({
        'Subject': [subj] * len(acf_epoch_all),  # Repite el sujeto para todas las filas
        'Combination': [combinacion] * len(acf_epoch_all),  # Repite la combinación para todas las filas
        "Epoch": range(1, len(acf_epoch_all) + 1),  # Número de épocas
        'ACF_epoch': acf_epoch_all,
        'ACW_50_epoch': acw_50_epoch_all,
        'ACW_0_epoch': acw_0_epoch_all,
        'ACW_50_epoch_mean': acw_50_epoch_all_mean,
        'ACW_0_epoch_mean': acw_0_epoch_all_mean,
        "promedio_ACW_50_electrodos": [promedio_ACW_50_electrodos] * len(acf_epoch_all),
        "promedio_ACW_0_electrodos": [promedio_ACW_0_electrodos] * len(acf_epoch_all)
        })

    #identificar acw_50


    #valores de estadística
        ## pero ver que valores son puto significativos de la ACF o de la ACW??



    isplot=False    

    # if isplot==True:
    #     # 1) Creamos un vector de lags en segundos,
    #     #    asumiendo que acf_val tiene tantos puntos como lags + 1.
    #     time_lags = np.arange(len(acf_val)) / sfreq

    #     # 2) Convertimos los índices (entero) de ACW_50 y ACW_0
    #     ACW_50_i = int(acw_50_lags)
    #     ACW_0_i = int(acw_0_lags)

    #     # 3) Figura
    #     plt.figure(figsize=(8, 4))

    #     # 4) Dibujamos la ACF en negro
    #     plt.plot(time_lags, acf_val, 'k', label='ACF')

    #     # 5) Acotamos el eje X a la duración total de la señal (o a lo que consideres)
    #     plt.xlim([0, time_lags[-1]])

    #     # 6) Rellenamos el área hasta ACW_50
    #     plt.fill_between(
    #         time_lags[:ACW_50_i + 1],
    #         acf_val[:ACW_50_i + 1],
    #         color='r', alpha=0.3, label='ACW-50 area'
    #     )

    #     # 7) Rellenamos el área hasta ACW_0
    #     plt.fill_between(
    #         time_lags[:ACW_0_i + 1],
    #         acf_val[:ACW_0_i + 1],
    #         color='m', alpha=0.3, label='ACW-0 area'
    #     )

    #     # 8) Título con valores de ACW
    #     plt.title(f'ACW-0 = {acw_0_s:.1f} s    ACW-50 = {acw_50_s:.1f} s')
    #     plt.xlabel('Lags (s)')
    #     plt.ylabel('Autocorrelation')
    #     plt.legend(loc='best')
    #     plt.show()

    return table_autocorrelation
    # return acf_epoch_all,acw_50_epoch_all, acw_0_epoch_all




In [174]:
table_autocorrelation= acf_epochs(epochs, isplot=False)

C:\Users\UCM\AppData\Local\Temp\ipykernel_9048\3536539863.py:11: FutureWarning: The current default of copy=False will change to copy=True in 1.7. Set the value of copy explicitly to avoid this warning
  data_epochs=epochs.copy().pick("eeg").get_data()


In [178]:
##codigo para agrupar todas las tablas

all_tables = []

for i in range(0,len(subject_names)):
    for h in range(0,len(combinaciones)):
        subj=subject_names[i]
        combinacion= combinaciones[h]
        path_epochs= fif_data / f"{subj}_epochs" / f"{subj}_{combinacion}-epo.fif"
        epochs = mne.read_epochs(path_epochs)
        table_autocorrelation= acf_epochs(epochs, isplot=False)
        all_tables.append(table_autocorrelation)

final_table = pd.concat(all_tables, ignore_index=True)


Reading g:\PROYECTO_SELF\Data_self\fif_data\S01_epochs\S01_S1 - S4-epo.fif ...
    Found the data of interest:
        t =       0.00 ...    9198.00 ms
        0 CTF compensation matrices available
Not setting metadata
19 matching events found
No baseline correction applied
0 projection items activated


C:\Users\UCM\AppData\Local\Temp\ipykernel_9048\3536539863.py:11: FutureWarning: The current default of copy=False will change to copy=True in 1.7. Set the value of copy explicitly to avoid this warning
  data_epochs=epochs.copy().pick("eeg").get_data()


Reading g:\PROYECTO_SELF\Data_self\fif_data\S01_epochs\S01_S1 - S5-epo.fif ...
    Found the data of interest:
        t =       0.00 ...    9198.00 ms
        0 CTF compensation matrices available
Not setting metadata
21 matching events found
No baseline correction applied
0 projection items activated


C:\Users\UCM\AppData\Local\Temp\ipykernel_9048\3536539863.py:11: FutureWarning: The current default of copy=False will change to copy=True in 1.7. Set the value of copy explicitly to avoid this warning
  data_epochs=epochs.copy().pick("eeg").get_data()


Reading g:\PROYECTO_SELF\Data_self\fif_data\S01_epochs\S01_S2 - S4-epo.fif ...
    Found the data of interest:
        t =       0.00 ...    9198.00 ms
        0 CTF compensation matrices available
Not setting metadata
17 matching events found
No baseline correction applied
0 projection items activated


...

C:\Users\UCM\AppData\Local\Temp\ipykernel_9048\3536539863.py:11: FutureWarning: The current default of copy=False will change to copy=True in 1.7. Set the value of copy explicitly to avoid this warning
  data_epochs=epochs.copy().pick("eeg").get_data()


Reading g:\PROYECTO_SELF\Data_self\fif_data\S06_epochs\S06_S3 - S4-epo.fif ...
    Found the data of interest:
        t =       0.00 ...    9198.00 ms
        0 CTF compensation matrices available
Not setting metadata
23 matching events found
No baseline correction applied
0 projection items activated


C:\Users\UCM\AppData\Local\Temp\ipykernel_9048\3536539863.py:11: FutureWarning: The current default of copy=False will change to copy=True in 1.7. Set the value of copy explicitly to avoid this warning
  data_epochs=epochs.copy().pick("eeg").get_data()


Reading g:\PROYECTO_SELF\Data_self\fif_data\S06_epochs\S06_S3 - S5-epo.fif ...
    Found the data of interest:
        t =       0.00 ...    9198.00 ms
        0 CTF compensation matrices available
Not setting metadata
24 matching events found
No baseline correction applied
0 projection items activated


C:\Users\UCM\AppData\Local\Temp\ipykernel_9048\3536539863.py:11: FutureWarning: The current default of copy=False will change to copy=True in 1.7. Set the value of copy explicitly to avoid this warning
  data_epochs=epochs.copy().pick("eeg").get_data()


In [180]:
final_table

,Subject,Combination,Epoch,ACF_epoch,ACW_50_epoch,ACW_0_epoch,ACW_50_epoch_mean,ACW_0_epoch_mean,promedio_ACW_50_electrodos,promedio_ACW_0_electrodos
0,S01,S1 - S4,1,"[[1.0, 0.9643618259278371, 0.882340758129411, ...","[0.02, 0.022, 0.136, 0.02, 0.026, 0.038, 0.038...","[0.676, 0.69, 0.926, 0.442, 0.446, 0.834, 0.82...",0.048102,0.748339,"[0.1008421052631579, 0.08231578947368423, 0.14...","[1.3477894736842104, 1.3209473684210529, 1.338..."
1,S01,S1 - S4,2,"[[1.0, 0.9567541395737114, 0.8559149870473575,...","[0.016, 0.016, 0.026, 0.034, 0.03, 0.034, 0.01...","[0.69, 0.686, 0.826, 1.078, 0.838, 0.82, 0.53,...",0.023559,0.405356,"[0.1008421052631579, 0.08231578947368423, 0.14...","[1.3477894736842104, 1.3209473684210529, 1.338..."
2,S01,S1 - S4,3,"[[1.0, 0.9786206576162684, 0.9299493435318699,...","[0.056, 0.034, 0.036, 0.058, 0.056, 0.052, 0.0...","[0.948, 0.604, 0.598, 0.872, 0.856, 0.54, 0.84...",0.046678,0.597932,"[0.1008421052631579, 0.08231578947368423, 0.14...","[1.3477894736842104, 1.3209473684210529, 1.338..."
3,S01,S1 - S4,4,"[[1.0, 0.9899768491836892, 0.9662969221780888,...","[0.238, 0.444, 0.402, 0.056, 0.042, 0.39, 0.34...","[2.17, 2.504, 2.132, 1.874, 1.616, 2.158, 1.95...",0.087932,1.626746,"[0.1008421052631579, 0.08231578947368423, 0.14...","[1.3477894736842104, 1.3209473684210529, 1.338..."
4,S01,S1 - S4,5,"[[1.0, 0.9873408230633265, 0.9566411543518696,...","[0.04, 0.042, 0.042, 0.034, 0.03, 0.044, 0.042...","[1.05, 1.218, 1.056, 0.876, 1.042, 1.21, 1.218...",0.031729,0.688034,"[0.1008421052631579, 0.08231578947368423, 0.14...","[1.3477894736842104, 1.3209473684210529, 1.338..."
...,...,...,...,...,...,...,...,...,...,...
789,S06,S3 - S5,20,"[[1.0, 0.9869102295034359, 0.9543366060899704,...","[0.112, 0.112, 0.04, 0.1, 0.032, 0.038, 0.052,...","[1.848, 2.128, 0.692, 1.614, 1.124, 1.148, 0.7...",0.087593,0.835492,"[0.09441666666666666, 0.08833333333333333, 0.0...","[1.2695833333333333, 1.3977500000000003, 1.217..."
790,S06,S3 - S5,21,"[[1.0, 0.9793835832916986, 0.929780373137263, ...","[0.056, 0.062, 0.058, 0.058, 0.062, 0.064, 0.0...","[1.048, 1.094, 0.898, 0.864, 1.048, 1.426, 0.5...",0.136746,1.277898,"[0.09441666666666666, 0.08833333333333333, 0.0...","[1.2695833333333333, 1.3977500000000003, 1.217..."
791,S06,S3 - S5,22,"[[1.0, 0.9714245636621274, 0.9020355283986697,...","[0.04, 0.038, 0.094, 0.042, 0.032, 0.03, 0.04,...","[2.74, 2.732, 2.554, 2.738, 0.408, 1.926, 2.17...",0.102949,0.914746,"[0.09441666666666666, 0.08833333333333333, 0.0...","[1.2695833333333333, 1.3977500000000003, 1.217..."
792,S06,S3 - S5,23,"[[1.0, 0.9670219901632972, 0.8874572750711319,...","[0.038, 0.038, 0.056, 0.272, 0.044, 0.044, 0.2...","[2.838, 2.67, 2.404, 2.916, 2.84, 2.684, 2.528...",0.089661,1.317254,"[0.09441666666666666, 0.08833333333333333, 0.0...","[1.2695833333333333, 1.3977500000000003, 1.217..."


In [181]:
final_table.to_csv(acw_analysis_path / f"final_table_acw_1-6.csv", index=False)

In [ ]:
##code to save the dataframe 

table_autocorrelation.to_csv(analysis_path / f{subj}_table_autocorrelation.csv", index=False)

In [ ]:
# code to read the dataframe

df_acf_loaded = pd.read_csv(analysis_path / f"acf_{layer_script}.csv")


In [186]:
df_acf_loaded

,Subject,Sensor,ACF_Zinnen,ACW_50_Zinnen,ACW_0_Zinnen,ACF_Woorden,ACW_50_Woorden,ACW_0_Woorden
0,sub-A2004,1,[ 1.00000000e+00 9.90504844e-01 9.62689459e-...,0.016667,0.395000,[ 1. 0.98957351 0.95919365 ... -1.02...,0.016667,0.343333
1,sub-A2004,2,[ 1. 0.99110624 0.96506121 ... -0.07...,0.018333,0.361667,[ 1. 0.99033796 0.96210035 ... 0.00...,0.016667,0.351667
2,sub-A2004,3,[ 1. 0.9913972 0.9661997 ... -0.21...,0.018333,0.398333,[ 1. 0.99071471 0.96354885 ... -0.08...,0.018333,0.351667
3,sub-A2004,4,[ 1. 0.99097473 0.96453989 ... -0.14...,0.016667,0.471667,[1. 0.99063727 0.96321899 ... 0.081492...,0.016667,0.328333
4,sub-A2004,5,[ 1. 0.99162543 0.96704988 ... -0.01...,0.018333,0.468333,[1. 0.99126763 0.96565904 ... 0.058078...,0.018333,0.266667
...,...,...,...,...,...,...,...,...
268,sub-A2004,269,[ 1. 0.98899351 0.95682109 ... -0.01...,0.015000,0.351667,[ 1. 0.98725621 0.95016538 ... -1.19...,0.013333,0.263333
269,sub-A2004,270,[ 1. 0.99171923 0.96734816 ... -0.01...,0.016667,0.041667,[1. 0.99178445 0.96756315 ... 0.375049...,0.016667,0.033333
270,sub-A2004,271,[ 1. 0.99140437 0.96611658 ... -0.32...,0.016667,0.035000,[1. 0.99110044 0.96495064 ... 0.241846...,0.016667,0.033333
271,sub-A2004,272,[1. 0.98998846 0.96066231 ... 0.008007...,0.016667,0.346667,[1. 0.98951551 0.95888772 ... 0.563044...,0.015000,0.041667


In [ ]:
#in case you want to add a new subjet to the dataframe its better to concatenate it 